Step 1. Load the image(s) as a numpy array with (h, w, rgb) shape as integers between 0 to 255

In [1]:
from numpy import asarray
from PIL import Image
import numpy as np

im = Image.open('targeted_contrast.jpg') #load image.jpg
i_a = asarray(im) #convert image to array

Step 2. Convert the range of pixel values from 0-255 to 0-1



In [2]:
i_a = i_a / 255.0

Step 3. Flatten the image to columns of RGB (3 columns)

Flattening the images merges all the layers (in this case, RGB layers) into one column each color channel.

In [3]:
img_rs = i_a.reshape(-1, 3)


Step 4. Centering the pixels around their mean



In [4]:
img_centered = img_rs - np.mean(img_rs, axis=0)


Step 5. Calculate the 3x3 covariance matrix using numpy.cov. The parameter rowvar is set as False because each column represents a variable, while rows contain the values.



In [5]:
img_cov = np.cov(img_centered, rowvar=False)
img_cov

array([[0.03288312, 0.03261066, 0.03402053],
       [0.03261066, 0.0330318 , 0.03480445],
       [0.03402053, 0.03480445, 0.0371824 ]])

Step 6. Calculate the eigenvalues (3x1 matrix) and eigenvectors (3x3 matrix) of the 3 x3 covariance matrix using numpy.linalg.eigh
Then, sort the eigenvalues and eigenvectors




In [6]:
eig_vals, eig_vecs = np.linalg.eigh(img_cov)
sort_perm = eig_vals[::-1].argsort()
eig_vals[::-1].sort()
eig_vecs = eig_vecs[:, sort_perm]
print("eigenvectors :",eig_vecs)
print('\n')
print("eigenvalues  :", eig_vals)

eigenvectors : [[-0.56294314 -0.74576875  0.35626395]
 [-0.56839384  0.0363944  -0.82195127]
 [-0.60001955  0.66521006  0.44437834]]


eigenvalues  : [1.02070722e-01 9.46086105e-04 8.05151156e-05]


 get eigenvector matrix 

In [7]:
m1 = np.column_stack((eig_vecs))
m1

array([[-0.56294314, -0.56839384, -0.60001955],
       [-0.74576875,  0.0363944 ,  0.66521006],
       [ 0.35626395, -0.82195127,  0.44437834]])

Step 7. Get a 3x1 matrix of eigenvalues multipled by a random variable drawn from a Gaussian distribution with mean=0 and sd=0.1 using numpy.random.normal



In [8]:
m2 = np.zeros((3, 1))
alpha = np.random.normal(0, 0.5)  # Increase the magnitude of alpha
m2[:, 0] = alpha * eig_vals[:]
add_vect = np.dot(eig_vecs, m2)
print(m2)

print("Note: alpha should only be drawn once per augmentation (not once per channel) \n- alpha: ", alpha)

[[-1.06450114e-01]
 [-9.86678374e-04]
 [-8.39696544e-05]]
Note: alpha should only be drawn once per augmentation (not once per channel) 
- alpha:  -1.0429054698135665


Step 8. Create and add the vector (add_vect) that we're going to add to each pixel



In [9]:
orig_img = i_a.reshape(im.size[1], im.size[0], 3).copy()



In [10]:
# Step 8: Create and add the vector (add_vect) that we're going to add to each pixel
# orig_img = i_a.reshape(im.size[1], im.size[0], 3).copy()

# Apply the augmentation to the original image
for idx in range(3):  # RGB
    orig_img[..., idx] = np.clip(orig_img[..., idx] + add_vect[idx], 0, 1)

# Ensure pixel values are in the valid range [0, 1]
orig_img = np.clip(orig_img, 0, 1)

# Convert back to the range [0, 255] for saving
orig_img = (orig_img * 255).astype(np.uint8)

# If you want to save the augmented image
augmented_image = Image.fromarray(orig_img)
augmented_image.save('augmented_image.jpg')

In [ ]:
import matplotlib.pyplot as plt

# ... (Previous code remains unchanged)

# If you want to save the augmented image
augmented_image = Image.fromarray(orig_img)
augmented_image.save('augmented_image.jpg')

# Load the original and augmented images for histogram comparison
original_image = Image.open('image.jpg')
augmented_image = Image.open('augmented_image.jpg')

# Plot the original and augmented images side by side with their RGB histograms
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Plot Original Image
axes[0, 0].imshow(original_image)
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

# Plot Augmented Image
axes[0, 1].imshow(augmented_image)
axes[0, 1].set_title('Augmented Image')
axes[0, 1].axis('off')

# Plot RGB Histograms for Original Image
for i in range(3):
    axes[1, 0].hist(original_image.getdata(band=i), bins=64, range=[0, 256], alpha=0.5, label=f'Channel {i}', color=f'C{i}')

axes[1, 0].set_title('RGB Histogram (Original)')
axes[1, 0].legend()

# Plot RGB Histograms for Augmented Image
for i in range(3):
    axes[1, 1].hist(augmented_image.getdata(band=i), bins=64, range=[0, 256], alpha=0.5, label=f'Channel {i}', color=f'C{i}')

axes[1, 1].set_title('RGB Histogram (Augmented)')
axes[1, 1].legend()

# Show the plot
plt.tight_layout()
plt.show()
